In [1]:
from data_loader import get_mix_instruct

prompts, references, ds_name = get_mix_instruct("train", 1000)

Dataset: mix-instruct_train_1000 found in cache, loading from cache ✅


In [5]:
from utility_functions.delift_se import get_delift_se_utility

utility = get_delift_se_utility(prompts, references, ds_name)

Utility: mix-instruct_train_1000_delift-se found in cache, loading from cache ✅


In [6]:
utility.shape

torch.Size([1000, 1000])